In [5]:
import pathlib
import numpy as np

from aiida import orm, load_profile
from aiida.engine import submit

from qe_tools import CONSTANTS

from aiida_quantumespresso_hp.workflows.hubbard import SelfConsistentHubbardWorkChain
from aiida_quantumespresso.data.hubbard_structure import HubbardStructureData

load_profile()

Profile<uuid='4fe00b7a32994cfca8b6cf43c2d22a53' name='jgeiger'>

# BaTiO3
Let's define the BTO structure to use as example for this tutorial.

In [19]:
cell = [
    [5.34198, -3.08419, 4.38073],
    [0.00000,  6.16839, 4.38073],
    [-5.3419, -3.08419, 4.38073],
]
cell = CONSTANTS.bohr_to_ang*np.array(cell)

structure = orm.StructureData(cell=cell)

structure.append_atom(position=np.dot([0.48674, 0.4867, 0.4867], cell),  symbols='Ti')
structure.append_atom(position=np.dot([0.51163, 0.5116, 0.0192], cell),  symbols='O' )
structure.append_atom(position=np.dot([0.51163, 0.0192, 0.5116], cell),  symbols='O' )
structure.append_atom(position=np.dot([0.01929, 0.5116, 0.5116], cell),  symbols='O' )
structure.append_atom(position=np.dot([0.99899, 0.9989, 0.9989], cell),  symbols='Ba')

## HubbardStructureData initialization
Let's initialize the HubbardStructureData with the BTO structure!

> Important: You only need to do these steps once! Later just keep track of this testing structure via the PK and reload it from the database so you don't get 100 BaTiO3 strucures in your storage. ^^

In [20]:
hubbard_data = HubbardStructureData(structure=structure)

In [21]:
hubbard_data

<HubbardStructureData: uuid: b49ea297-51e0-40b7-bdd6-10a53f3e1bb6 (unstored)>

## Initializing the on-site Hubbard
Let's initialize the on-site Hubbard parameter for the titanium atom.

In [22]:
hubbard_data.initialize_onsites_hubbard('Ti', '3d', 5.0) # index 0 for Ti ==> Hubbard U

Here how it is stored in the class.

In [23]:
hubbard_data.hubbard_parameters

[[0, '3d', 0, '3d', 5.0, [0, 0, 0], 'dudarev']]

## Initializing the inter-site Hubbard
Let's initialize the inter-site Hubbard parameter for the titanium and oxygen atoms.

In [24]:
hubbard_data.initialize_intersites_hubbard('Ti', '3d', 'O', '2p', 0.0001, number_of_neighbours=7)

The parameters are saved in the property `hubbard_parameters` as a list.

In [25]:
hubbard_data.hubbard_parameters

[[0, '3d', 0, '3d', 5.0, [0, 0, 0], 'dudarev'],
 [0, '3d', 3, '2p', 0.0001, [0, 0, 0], 'dudarev'],
 [0, '3d', 1, '2p', 0.0001, [0, 0, 0], 'dudarev'],
 [0, '3d', 2, '2p', 0.0001, [0, 0, 0], 'dudarev'],
 [0, '3d', 1, '2p', 0.0001, [0, 0, 1], 'dudarev'],
 [0, '3d', 2, '2p', 0.0001, [0, 1, 0], 'dudarev'],
 [0, '3d', 3, '2p', 0.0001, [1, 0, 0], 'dudarev'],
 [0, '3d', 3, '2p', 0.0001, [0, 0, -1], 'dudarev']]

In [17]:
hubbard_data.store()

<HubbardStructureData: uuid: 121783bc-8579-4b3a-986b-2740cc20fec0 (pk: 2502)>

In [ ]:
hubbard_data.pk

## Running the `SelfConsistentHubbardWorkChain`

In [6]:
hubbard_data = orm.load_node(2502)  # I load the node from the database instead of always regenerating it
print(hubbard_data.get_quantum_espresso_hubbard_card())

HUBBARD	ortho-atomic
 U	Ti-3d	5.0
 V	Ti-3d	O-2p	1	4	0.0001
 V	Ti-3d	O-2p	1	2	0.0001
 V	Ti-3d	O-2p	1	3	0.0001
 V	Ti-3d	O-2p	1	72	0.0001
 V	Ti-3d	O-2p	1	83	0.0001
 V	Ti-3d	O-2p	1	114	0.0001
 V	Ti-3d	O-2p	1	69	0.0001



In [20]:
pw_code = orm.load_code('qe-dev-pw@lumi-small')
hp_code = orm.load_code('qe-dev-hp@lumi-small')

builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=hubbard_data,
    protocol='fast',
    overrides=pathlib.Path('hubbard_overrides.yaml')
)

builder.skip_first_relax = True

In [25]:
third_submit = submit(builder)
print(third_submit)

uuid: d533fccf-1722-4785-b526-2c6db43a6753 (pk: 2642) (aiida.workflows:quantumespresso.hp.hubbard)


In [19]:
first_submit_pk = 2234
second_submit_pk = 2448
third_submit_pk = 2642

In [26]:
%%bash
verdi process show 2234
printf '=%.0s' {1..100}
echo
verdi process status 2234
printf '=%.0s' {1..100}
echo
verdi process report 2234
printf '=%.0s' {1..100}
echo

Property     Value
-----------  ---------------------------------------------------------------------
type         SelfConsistentHubbardWorkChain
state        Finished [402] The PwRelaxWorkChain sub process failed in iteration 2
pk           2234
uuid         8a7af431-03dd-44f4-a84c-a17e55ead812
label
description
ctime        2023-02-16 10:03:47.493781+01:00
mtime        2023-02-16 10:21:15.871575+01:00

Inputs                               PK    Type
-----------------------------------  ----  --------------------
hubbard
    hp
        code                         2183  InstalledCode
        qpoints                      2223  KpointsData
        parameters                   2224  Dict
        settings                     2225  Dict
    clean_workdir                    2226  Bool
    max_iterations                   2227  Int
    parallelize_atoms                2228  Bool
relax
    base
        pw
            pseudos
                Ti                   14    UpfData
                O

In [7]:
%%bash
verdi process show 2448
printf '=%.0s' {1..100}
echo
verdi process status 2448
printf '=%.0s' {1..100}
echo
verdi process report 2448
printf '=%.0s' {1..100}
echo

Property     Value
-----------  ---------------------------------------------------------------------
type         SelfConsistentHubbardWorkChain
state        Finished [402] The PwRelaxWorkChain sub process failed in iteration 2
pk           2448
uuid         767c3be7-8042-4e79-9fe6-8ec2af85655e
label
description
ctime        2023-02-21 14:30:56.614402+01:00
mtime        2023-02-21 15:22:09.913790+01:00

Inputs                               PK    Type
-----------------------------------  ----  --------------------
hubbard
    hp
        code                         2183  InstalledCode
        qpoints                      2437  KpointsData
        parameters                   2438  Dict
        settings                     2439  Dict
    clean_workdir                    2440  Bool
    max_iterations                   2441  Int
    parallelize_atoms                2442  Bool
relax
    base
        pw
            pseudos
                Ti                   14    UpfData
                O

In [3]:
%%bash
verdi process show 2642
printf '=%.0s' {1..100}
echo
verdi process status 2642
printf '=%.0s' {1..100}
echo
verdi process report 2642
printf '=%.0s' {1..100}
echo

Property     Value
-----------  ------------------------------------
type         SelfConsistentHubbardWorkChain
state        Finished [0]
pk           2642
uuid         d533fccf-1722-4785-b526-2c6db43a6753
label
description
ctime        2023-02-26 08:56:37.499787+01:00
mtime        2023-02-26 09:57:58.079332+01:00

Inputs                               PK    Type
-----------------------------------  ----  --------------------
hubbard
    hp
        code                         2183  InstalledCode
        qpoints                      2516  KpointsData
        parameters                   2517  Dict
        settings                     2518  Dict
    clean_workdir                    2519  Bool
    max_iterations                   2639  Int
    parallelize_atoms                2640  Bool
relax
    base
        pw
            pseudos
                Ti                   14    UpfData
                O                    74    UpfData
                Ba                   45    UpfData
     